In [ ]:
# Luồng hoạt động trong RTK Query + Custom Base Query

# 1. Quy trình hoạt động tuần tự 
- Hãy tưởng tượng luồng đi từ lúc "gọi hook [use...] trong Component --> đến khi nhận được dữ liệu hiển thị lên UI màn hình"

**Bước 1: Trigger (Kích hoạt từ Component). Bạn gọi hook trong Component.**

In [ ]:
// Bạn truyền object { page: 1 } vào hook
const { data, isLoading } = useGetProductsQuery({ page: 1 });

**Bước 2: Định nghĩa API Endpoint:**
- RTK Query nhận tham số `{ page: 1 }`
- Tìm đến Endpoint `getProducts` đã khai báo trong `productApi`
- Nó gọi hàm `query` bên trong Endpoint đó

In [ ]:
// params lúc này là { page: 1 }
query: (params) => ({
    url: "/products",
    method: "GET",
    params: params, // { page: 1 }
})

// --> Hàm trả về một object cấu hình (chúng ta gọi là `args`)

**Bước 3: Xử lý trung gian Base Query**
- Object `args` ở Bước 2 được chuyển tới hàm `baseQuery`. Tại đây logic chuyển đổi sang Axios diễn ra:
    1. Nhận `args`: `{url: '/products', method: "GET", params: {page: 1}}`
    2. Tạo `config`: Nó map field từ `args` sang `config` của Axios:
        - `config.url` = `'/product'`
        - `config.method` = `'GET'`
        - `config.params` = `{page: 1}`

    3. Gọi `httpRequest`: Thực hiện gọi API lên server thực tế

**Bước 4: Server Response (Phản hồi từ Server). Server xử lý và trả về dữ liệu (hoặc lỗi)**
- Ví dụ thành công: Trả về mảng danh sách sản phẩm

**Bước 5: Chuẩn hoá dữ liệu: `baseQuery` nhận kết quả từ `httpRequest`**
- Nếu thành công: Nó bọc dữ liệu vào object `{ data: ... }`
- Nếu thất bại (`catch`): Nó bọc lỗi vào object `{ error: ... }`

=> Lưu ý: RTK Query bắt buộc `baseQuery` phải trả về đúng định dạng `{ data }` hoặc `{ error }` thì nó mới hiểu

**Bước 6: Caching & Update Store**: RTK Query nhận kết quả từ `baseQuery`
- Nó lưu dữ liệu vào Redux Store (Cache) dựa trên `tagType` ("Products")
- Nó cập nhật trạng thái `isLoading` từ `true` => `false`

**Bước 7: Re-render Component:**
- Component nhận thấy store thay đổi, nó re-render với dữ liệu mới nhất trong biến `data`

# Ý nghĩa & cách sử dụng các tham số trong `query`

In [ ]:
query: (inputData) => ({
    url: "...",
    method: "...",
    params: inputData, // hoặc data: inputData
})

## A. Tham số đầu vào của hàm `query`:
- Là dữ liệu được truyền trực tiếp vào Hook khi gọi ở component
    - Ví dụ: `useCreateProductMutation(newProductData)`
    - Thì: `inputData` trong `query: (inputData)...` chính là `newProductData`

## B. Ví dụ minh hoạ sự khác biệt giữa `params` và `data`

### TH1: `getProduct` (Dùng `params`)

In [ ]:
// Code:
query: (filter) => ({
    url: "/products",
    method: "GET",
    params: filter // filter = { category: 'shoes' }
})
// Kết quả gọi API thực tế:
// GET /products?category=shoes
// (Không có Body)

### TH 2: `createProduct` (Dùng data)

In [ ]:
// Code:
query: (productInfo) => ({
    url: "/products",
    method: "POST",
    data: productInfo // productInfo = { name: 'Giày', price: 100 }
})
// Kết quả gọi API thực tế:
// POST /products
// Body: { "name": "Giày", "price": 100 }

## TÓM LẠI:
1. Luồng đi: Component -> Hook -> `query` (tạo config) -> `baseQuery` (gọi Axios) -> Server -> `baseQuery` (trả về `{ data }` -> Store -> Component)
2. `params`: Dùng cho dữ liệu trên thanh địa chỉ URL (GET)
3. `data`: Dũng cho dữ liệu ẩn bên trong gói tin (POST/PUT/PATCH)